In [1]:
import dimod
import neal
import numpy as np
import itertools
import math
import copy

In [2]:
# --- エクセルファイルからデータ読み込み ---
import openpyxl

def CellList(excel_file, sheet_num=0):
    """
    入力：エクセルファイルのpath
    出力：[[ID, NAME, CLASS, TEACHER, PLACE], 
          [ID, NAME, CLASS, TEACHER, PLACE],
             ... 
          [ID, NAME, CLASS, TEACHER, PLACE]]
    """
    def get_sheet(excel_file):
        wb = openpyxl.load_workbook(excel_file)
        sheet_name = wb.sheetnames[sheet_num] # Sheet1に時間割が書かれていると想定
        return wb[sheet_name]
    sheet = get_sheet(excel_file)
    cell_list = list(sheet.values)
    return cell_list[1:]

def ClassDict(cell_list):
    """
    入力：cell_list
    出力：class_dict = {ID : {'NAME':'J1', 'CLASS':['1A'], 'TEACHER':['a'], 'PLACE':['1A']},
                       ID : {'NAME':'J2', 'CLASS':['1A'], 'TEACHER':['b'], 'PLACE':['1A']},
                            ...
                       ID : {'NAME':'PE3', 'CLASS':['1A','1B'], 'TEACHER':['g'], 'PLACE':['体育館']}}
    """
    class_dict = {}
    total = len(cell_list)
    for i in range(total):
        class_dict[i] = {}
        class_dict[i]['NAME'] = cell_list[i][1]
        if cell_list[i][2]:
            class_dict[i]['CLASS'] = list(cell_list[i][2].split(','))
        else:
            class_dict[i]['CLASS'] = []
        class_dict[i]['TEACHER'] = list(cell_list[i][3].split(','))
        if cell_list[i][4]:
            class_dict[i]['PLACE'] = list(cell_list[i][4].split(','))
        else:
            class_dict[i]['PLACE'] = []
    return class_dict

def Adjacent(class_dict):
    """
    入力：class_dict
    出力：adjacent (隣接、重なってはいけない授業)
    """
    total = len(class_dict) # cell_listには全ての授業の情報が入っているはず
    adjacent = {}
    for i in range(total):
        adjacent[i] = []
        for j in range(total):
            if i != j:
                c1 = class_dict[i]
                c2 = class_dict[j]
                # CLASS or TEACHER or PLACEが同じ(もしくは被ってるものがある)
                if set(c1['CLASS'])&set(c2['CLASS']) or set(c1['TEACHER'])&set(c2['TEACHER']) or set(c1['PLACE'])&set(c2['PLACE']):
                    adjacent[i].append(j)
    return adjacent

# 前処理

def Preprocessing(preprocessing,koma_data,class_dict,class_dict2):

    """
    preprocessing：[ID, c] 
    """
    
    total = len(class_dict) # 今回やりたい学年のデータ
    total2 = len(class_dict2) # 前回の学年のデータ
    for i in range(total):
        for j in range(total2):
            c1 = class_dict[i]
            c2 = class_dict2[j]
            # TEACHER or PLACEが同じ(もしくは被ってるものがある)
            if set(c1['TEACHER'])&set(c2['TEACHER']) or set(c1['PLACE'])&set(c2['PLACE']):
                koma=koma_data[j] 
                preprocessing.append([i,koma])# 科目iをkomaに入れないようにする

    



In [3]:
"""
対応関係
q : weekly
47 : total
"""
def Hamiltonian(weekly, total, gnum,
                preprocessing, #第0項
                adjacent,  # 第1項
                convenience,  # 第3項
                renzoku_koma, renzoku_ID, # 第4項
                table, one_per_day,  # 第5項
                joint,  # 第6項
                gen_list, one_per_gen,   # 第7項
                renzoku_2koma, not_renzoku_ID,  # 第8項
                w0=1.0,w1=1.0, w2=1.0, w3=1.0, w4=1.0, w5=1.0, w6=1.0, w7=1.0, w8=1.0  # 重み
                ):
    """
    ハミルトニアンを計算してQUBOとして返す
    q: 1週間のコマの数、つまりweekly
    alpha: 制約の強さ
    """
    constant = 0
    A = np.zeros((total*weekly, total*weekly))
   
    #第0項　前処理(他の学年とかぶらないようにする)
    for pre in preprocessing:
        # preprocessing：[ID, c, w(>0)]
        i, a= pre[0], pre[1]
        k = weekly * i + a
        A[k, k] += w0  
   
    
    
    # 第1項：問題を表す
    for i in range(total):
        for j in adjacent[i]:
            if sorted([i, j]) in joint:
                continue
            for a in range(weekly):
                k1 = weekly * i + a
                k2 = weekly * j + a
                A[k1, k2] += w1
    # 第2項：制約条件（１つの科目に１つのコマしか割り当てない）
    plus2 = 0
    for i in range(total):
        if len(class_dict[i]['CLASS']) > 1:
            plus2 = 0
        else:
            plus2 = 0
        for a in range(weekly):
            k1 = weekly * i + a
            for b in range(weekly):
                k2 = weekly * i + b
                if k1 == k2: # 1次の項は対角成分に入れる
                    A[k1, k2] -= (w2 + plus2)
                else:
                    A[k1, k2] += (w2 + plus2)
    # 第3項：制約条件（教員の都合を反映）
    for con in convenience[gnum]:
        # convenience：[ID, c, w(>0)]
        i, a, w = con[0], con[1], con[2]
        k = weekly * i + a
        A[k, k] += w*w3  # w > 0 : 避けたい,  w < 0 : 入りたい
        if w < 0:
            constant -= w*w3
    # 第4項：制約条件（2時間続きにしたい授業）
    for IDs in renzoku_ID[gnum]:
        constant += w4
        for renzoku in renzoku_koma:
            k1 = weekly * IDs[0] + renzoku[0]
            k2 = weekly * IDs[1] + renzoku[1]
            A[k1, k2] -= w4
    # 第5項：制約条件（各科目は1日1コマ）
    for IDs in one_per_day[gnum]:
        for komas in table:
            for i in IDs:
                for j in IDs:
                    for a in komas:
                        for b in komas:
                            k1 = weekly * i + a
                            k2 = weekly * j + b
                            if k1 != k2:
                                A[k1, k2] += w5
    # 第6項：制約条件（2クラスでの合同授業）
    """
    TODO: 第1項との絡みをどうするか、(同じ先生、同じ場所の科目はそもそも繋ぐようにしている)
    合同にしたい授業はグラフ上でそもそも繋がないか。
    また、3クラス以上で可能なのか
    """
    for IDs in joint[gnum]:
        constant += w6
        for a in range(weekly):
            k1 = weekly * IDs[0] + a
            k2 = weekly * IDs[1] + a
            A[k1, k2] -= w6
    # 第7項：制約条件（各科目が同じ時間帯に固まらないようにする）
    for IDs in one_per_gen[gnum]:
        for komas in gen_list:
            for i in IDs:
                for j in IDs:
                    for a in komas:
                        for b in komas:
                            k1 = weekly * i + a
                            k2 = weekly * j + b
                            if k1 != k2:
                                A[k1, k2] += w7
    # 第8項：制約条件（1教員が3コマ連続にならないようにする）
    for ID_list in not_renzoku_ID[gnum]:
        ID2 = list(itertools.combinations(ID_list, 2))
        for IDs in ID2:
            for koma2 in renzoku_2koma:
                k1 = weekly * IDs[0] + koma2[0]
                k2 = weekly * IDs[1] + koma2[1]
                A[k1, k2] += w8
                
    # dimodソルバー用のQUBO形式に変換
    Q = {}
    for k1 in range(total*weekly):
        for k2 in range(total*weekly):
            if A[k1, k2] != 0:
                Q[(k1, k2)] = A[k1, k2]
    return Q, constant

In [4]:
""" 制約破り判断用の関数 """
# 第1項：グラフで繋がれた授業が被っていない
def is_satisfied_1(adjacent, joint, koma_data):
    broken = []
    satisfied = True
    for i in adjacent:
        for j in adjacent[i]:
            if koma_data[i] == koma_data[j] and tuple(sorted([i, j])) not in joint:
                satisfied = False
                broken.append((i, j))
    if broken:
        print("制約1破り：重なってはいけないのに重なっている授業IDの組")
        print(broken)
    return satisfied

# 第2項：１つの科目に１つのコマしか割り当てない
def is_satisfied_2(koma_data, total):
    if len(koma_data) == total:
        return True
    else:
        print("制約2破り：コマが割り当てられていない授業IDの個数")
        print(str(total - len(koma_data)) + "個")
        return False

# 第3項：教員の都合を反映
def is_satisfied_3(convenience, koma_data):
    satisfied = True
    broken = []
    for con in convenience:
        ID, koma = con[0], con[1]
        if koma_data[ID] == koma:
            satisfied = False
            broken.append("ID: {}, コマ: {}".format(ID, koma))
    if broken:
        print("制約3破り：都合が反映されていない授業IDとそのコマ")
        print(broken)
    return satisfied

# 第4項：2時間続きにしたい授業
def is_satisfied_4(renzoku_ID, renzoku_koma, koma_data):
    satisfied = True
    broken = []
    for IDs in renzoku_ID:
        #if abs(koma_data[IDs[0]] - koma_data[IDs[1]]) != 1:
        if sorted([koma_data[IDs[0]], koma_data[IDs[1]]]) not in renzoku_koma:
            satisfied = False
            broken.append((IDs[0], IDs[1]))
    if broken:
        print("制約4破り：2時間続きになっていない授業IDの組")
        print(broken)
    return satisfied

# 第5項：各科目は1日1コマ
def is_satisfied_5(one_per_day, table, koma_data):
    satisfied = True
    broken = set()
    for IDs in one_per_day:
        for day in table:
            times = 0
            for ID in IDs:
                if koma_data[ID] in day:
                    times += 1
            if times > 1:
                broken.add(tuple(IDs))
                satisfied = False
    if broken:
        print("制約5破り：1日に1コマ以上入っている授業群")
        print(broken)
    return satisfied

# 第6項：2クラス合同
def is_satisfied_6(joint, koma_data):
    broken = []
    for j in joint:
        if koma_data[j[0]] != koma_data[j[1]]:
            broken.append(j)
    if broken:
        print("制約6破り：2クラス合同になっていない授業群")
        print(broken)
    return not broken

# 第7項：各科目が同じ時間帯に固まらないようにする
def is_satisfied_7(one_per_gen, gen_list, koma_data):
    satisfied = True
    broken = set()
    for IDs in one_per_gen:
        for day in gen_list:
            times = 0
            for ID in IDs:
                if koma_data[ID] in day:
                    times += 1
            if times > 1:
                broken.add(tuple(IDs))
                satisfied = False
    if broken:
        print("制約7破り：同じ時間帯に入っている授業群")
        print(broken)
    return satisfied

# 第8項：1教員が3コマ連続にならないようにする
def is_satisfied_8(table, not_renzoku_ID, renzoku_3koma, koma_data):
    broken = []
    for IDs in not_renzoku_ID:
        ID3s = list(itertools.combinations(IDs, 3))
        for ID in ID3s:
            komas = [koma_data[ID[0]], koma_data[ID[1]], koma_data[ID[2]]]
            komas.sort()
            if komas[1] - komas[0] == 1 and komas[2] - komas[1] == 1:
                for t in table:
                    if komas[0] in t and komas[1] in t and komas[2] in t:
                        broken.append(ID)
                        break
    if broken:
        print("制約8破り：1教員が3コマ連続になっている授業群")
        print(broken)
    return not broken


def constraint(adjacent, total, convenience, renzoku_ID, renzoku_koma, one_per_day, one_per_gen,joint, table, not_renzoku_ID, renzoku_3koma, koma_data):
    satisfied = True
    if not is_satisfied_2(koma_data, total):
        print("制約2破りのため、他の制約に関しては不明")
        satisfied = False
    elif not is_satisfied_1(adjacent, joint, koma_data):
        print("制約1破りのため、他の制約に関しては不明")
        satisfied = False
        if not is_satisfied_6(joint, koma_data):
            satisfied = False
    else:
        if not is_satisfied_3(convenience, koma_data):
            satisfied = False
        if not is_satisfied_4(renzoku_ID, renzoku_koma, koma_data):
            satisfied = False
        if not is_satisfied_5(one_per_day, table, koma_data):
            satisfied = False
        if not is_satisfied_7(one_per_gen, gen_list, koma_data):
            satisfied = False
        if not is_satisfied_8(table, not_renzoku_ID, renzoku_3koma, koma_data):
            satisfied = False
        if not is_satisfied_6(joint, koma_data):
            satisfied = False
    if satisfied:
        print("成功")

def ok_1_2(adjacent, total, koma_data):
    if not is_satisfied_2(koma_data, total):
        return '制約2×'
    elif not is_satisfied_1(adjacent, joint, koma_data):
        return '制約1×'
    else:
        return '◯'

In [5]:
# その他関数群
def search(table, koma):
    for i in range(len(table)):
        j = 0
        for t in table[i]:
            if koma == t:
                return i, j
            j += 1
            
def ClassName(class_dict):
    class_name = set()
    for i in range(total):
        for name in class_dict[i]['CLASS']:
            class_name.add(name)
    class_list = list(class_name)
    class_list.sort()
    return class_list

def TeacherName(class_dict):
    teacher_name = set()
    for i in range(total):
        for name in class_dict[i]['TEACHER']:
            teacher_name.add(name)
    teacher_list = list(teacher_name)
    teacher_list.sort()
    return teacher_list

def ClassTable(koma_data, class_dict, class_name, table):
    class_table = {}
    for name in class_name:
        # tableの形によるかも
        class_table[name] = [[None] * len(table) for i in range(len(table[0]))]
    total = len(class_dict)
    #for ID in range(total):
    for ID in koma_data:
        x, y = search(table, koma_data[ID])
        for name in class_dict[ID]['CLASS']:
            # ここでx, yをひっくり返している
            class_table[name][y][x] = class_dict[ID]['NAME'] + ',' + ','.join(class_dict[ID]['TEACHER']) + ',' + ','.join(class_dict[ID]['PLACE'])
    return class_table

def TeacherTable(koma_data, class_dict, teacher_name, weekly):
    teacher_table = {}
    for name in teacher_name:
        teacher_table[name] = [[None] * weekly]
    total = len(class_dict)
    #for ID in range(total):
    for ID in koma_data:
        order = koma_data[ID]
        for name in class_dict[ID]['TEACHER']:
            teacher_table[name][0][order] = class_dict[ID]['NAME'] + ',' + ','.join(class_dict[ID]['CLASS'])
    return teacher_table

# --- 時間割を別のExcelファイルに出力 ---
def write_list_2d(sheet, list2d, start_row, start_col):
    """
    2次元配列を'そのまま見たまんま'Excelファイルのセルに書き込む
    入力：sheet, 2次元配列, 左上セルの行, 左上セルの列
    """
    for y, row in enumerate(list2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                      column=start_col + x,
                      value=list2d[y][x])
            
def MakeExcelFile(output_file, class_dict, koma_data, table, weekly):
    # 書き込みに必要なデータ
    class_name = ClassName(class_dict)
    class_table = ClassTable(koma_data, class_dict, class_name, table)
    teacher_name = TeacherName(class_dict)
    teacher_table = TeacherTable(koma_data, class_dict, teacher_name, weekly)

    # 書き込み用
    wb = openpyxl.Workbook()

    # 曜日、何限目などの体裁を整える
    weekday = ['月', '火', '水', '木', '金', '土']
    gen = [[1], [2], [3], [4], [5], [6], [7], [8], [9]]

    # sheet1
    sheet1 = wb.create_sheet('クラス別の時間割')
    for i in range(len(class_name)):
        # クラス名
        write_list_2d(sheet1, [[class_name[i]]], 1+i*(len(table[0])+2), 1)
        # 曜日
        write_list_2d(sheet1, [weekday[:len(table)]], 1+i*(len(table[0])+2), 2)
        # 何限
        write_list_2d(sheet1, gen[:len(table[0])], 2+i*(len(table[0])+2), 1)
        # 時間割
        write_list_2d(sheet1, class_table[class_name[i]], 2+i*(len(table[0])+2),2)
        i += 1
    # sheet2
    sheet2 = wb.create_sheet('先生別の時間割')
    # 曜日・何限
    allkoma = [[]]
    for i in range(len(table)):
        for j in range(len(table[i])):
            string = weekday[i] + str(gen[j][0])
            allkoma[0].append(string)
    write_list_2d(sheet2, allkoma, 1, 2)
    # 先生の時間割
    for i in range(len(teacher_name)):
        write_list_2d(sheet2, [['先生']], 1, 1)
        # 先生の名前
        write_list_2d(sheet2, [[teacher_name[i]]], 2+i, 1)
        # 時間割
        write_list_2d(sheet2, teacher_table[teacher_name[i]], 2+i, 2)

    wb.save(output_file)

In [6]:
# 第3項：先生の都合を表す
"""
convenience：[ID, c, w(>0)]
先生の都合を表す2次元配列（授業IDはコマcに入りたくない）

下の例：4つの授業（ID:0,1,2,9）を持つa先生はコマ0, 3に入りたくない
"""
#convenience = [# a先生はコマ0, 3に入りたくない
#               [0, 0, 1],
#               [1, 0, 1],
#               [2, 0, 1],
#               [9, 0, 1],
#               [0, 3, 1],
#               [1, 3, 1],
#               [2, 3, 1],
#               [9, 3, 1],
#               # f先生はコマ0, 3に入りたい
#                [6, 0, -1],
#                [15, 0, -1],
#                [24, 0, -1],
#                [6, 3, -1],
#                [15, 3, -1],
#                [24, 3, -1],
#                ]

#convenience = [
    # ボブは月曜日に入りたくない
#    [13,0,1],[13,1,1],[13,2,1],[13,3,1],[13,4,1],
#    [14,0,1],[14,1,1],[14,2,1],[14,3,1],[14,4,1],
#    [38,0,1],[38,1,1],[38,2,1],[38,3,1],[38,4,1],
#    [39,0,1],[39,1,1],[39,2,1],[39,3,1],[39,4,1],
#    [62,0,1],[62,1,1],[62,2,1],[62,3,1],[62,4,1],
#    [63,0,1],[63,1,1],[63,2,1],[63,3,1],[63,4,1],
#     [87,0,1],[87,1,1],[87,2,1],[87,3,1],[87,4,1],
#     [88,0,1],[88,1,1],[88,2,1],[88,3,1],[88,4,1],
    # 佐藤は水1,2,3×
#     [0,10,1],[0,11,1],[0,12,1],
#     [1,10,1],[1,11,1],[1,12,1],
#     [2,10,1],[2,11,1],[2,12,1],
#     [25,10,1],[25,11,1],[25,12,1],
#     [26,10,1],[26,11,1],[26,12,1],
#     [27,10,1],[27,11,1],[27,12,1],
#     [49,10,1],[49,11,1],[49,12,1],
#     [50,10,1],[50,11,1],[50,12,1],
#     [51,10,1],[51,11,1],[51,12,1],
#     [74,10,1],[74,11,1],[74,12,1],
#     [75,10,1],[75,11,1],[75,12,1],
#     [76,10,1],[76,11,1],[76,12,1],
#]

# # 1,2年 道徳1,2年ごとに合同あり+会議
# ids_komas_list = [ # [[ID,ID,..], [koma,koma,koma,...]]
#     #[[122,123,148,149,177,178,203,204],[0,1,2,3,4,5,6,14,15,16,17,18,26,27,28,29,30,31,32]], # so7, 月水金×
#     #[[110,111,140,141,163,164,165,166],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], # m14, 月火水×
#     #[[117,143,172,198],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], # sc3, 月火水×
#     [[17,43,69,94],[0,1,2,3,4,5,6,14,15,16,17,18,19,20,21,22,23,24,25]], # sc10, 月水木×
#     [[26,78],[0,1,2,3,4,5,6,14,15,16,17,18]], # p7, 月水×
#     [[129,184],[19,20,21,22,23,24,25]], # p8, 木×
#     [[25,77],[0,1,2,3,4,5,6]], # p9, 月×
# ]
# for [ids, komas] in ids_komas_list:
#     for id1 in ids:
#         for koma in komas:
#             convenience.append([id1, koma, 1])

#福田先生は1限に入れない,一つしかないものは1限に入れない
convenience = [
    [[63,0,1],[64,0,1],[105,0,1],[106,0,1],
    [63,5,1],[64,5,1],[105,5,1],[106,5,1],
    [63,10,1],[64,10,1],[105,10,1],[106,10,1],
    [63,15,1],[64,15,1],[105,15,1],[106,15,1],
    [63,20,1],[64,20,1],[105,20,1],[106,20,1],],
    [],
]  

#１コマ分しかない教科は一限に入れない
#convenience = [[],[]]

In [7]:
# 第4項
# 連続しているコマ
#renzoku_koma = [[0, 1],[1, 2],[3, 4],[4, 5],[6, 7],[7, 8],]

renzoku_koma = [[0,1],[1,2],[3,4],[5,6],[6,7],[8,9],[10,11],[11,12],[13,14],[15,16],[16,17],[18,19],[20,21],[21,22],[23,24],]

#renzoku_koma = [[0,1],[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[11,12],[12,13],[14,15],[15,16],[16,17],[17,18],[19,20],[20,21],[21,22],[22,23],[23,24],[24,25],[26,27],[27,28],[28,29],[29,30],[30,31],[31,32]]
# 連続させたい授業
# [Ma1(ID:3), Ma2(ID:4)], [Mb1(ID:12), Mb2(ID:13)], [Ja1(ID:0), Ja2(ID:1)]を連続させたい
#renzoku_ID = [[3, 4], [12, 13], [0, 1]]
#renzoku_ID = [[15,16], [40,41], [64,65]]
# # 体育1A1B, 1C1D
# renzoku_ID = [[25,77],
#               [26,78],
#               [27,79]
# ]

#家庭科は連続にする
renzoku_ID = [[[19,20],[42,43],[65,66],[86,87],[107,108],[128,129],[149,150],[172,173],[195,196],],[],]
renzoku_ID = [[],[]]

In [8]:
# 第5項
# 1日1コマのみ入ってほしい科目群
# [Ja1(ID:0), Ja2(ID:1), Ja3(ID:2)], [Ea1(ID:6), Ea2(ID:7)], ...
#one_per_day = [[0,1,2], [3,4,5], [6,7],
#               [9,10,11], [12,13,14], [15,16],
#               [18,19,20], [21,22,23], [24,25],
#              ]
# class4:
#"""
one_per_day = [[
#     [10,11,12,13,14],[35,36,37,38,39],[59,60,61,62,63],[84,85,86,87,88], # 英語
#     [0,1,2,3,4],[25,26,27,28,29],[49,50,51,52,53],[74,75,76,77,78], # 国語
#     [5,6,7,8,9],[30,31,32,33,34],[54,55,56,57,58],[79,80,81,82,83], # 数学
#     [15,16,17,18],[40,41,42,43],[64,65,66,67],[89,90,91,92], # 理科
    [19,20,21,22],[44,45,46,47],[68,69,70,71],#[93,94,95,96], # 社会
]
]
# one_per_day = [
#     [0,1,2,3],[33,34,35,36],[52,53,54,55],[84,85,86,87], # 1年国語
#     [103,104,105,106,107],[135,136,137,138,139],[158,159,160,161,162],[190,191,192,193,194], # 2年国語
#     [4,5,6,7,8],[4,5,6,37,38],[56,57,58,59,60],[56,57,58,88,89], # 1年数学
#     [108,109,110,111],[108,109,140,141],[163,164,165,166],[163,164,195,196], # 2年数学
#     [9,10,11,12,13],[9,10,11,12,39],[61,62,63,64,65],[61,62,63,64,90], # 1年英語
#     [112,113,114,115,116],[112,113,114,115,142],[167,168,169,170,171],[167,168,169,170,197], # 2年英語
    
# ]
#５教科は一日一コマ とりあえず体育
one_per_day = [[[199,200],[201,202],[203,204],[205,206],[207,208],],[],]
#one_per_day = [[],[]]

In [9]:
# 第6項
# 2クラス合同 小石川1,2年 合同なし用
#j = [4,5,6,9,10,11,12,22,23,24,25,26,27,70,71,72,75,76,77,78,88,89,90,91,92,93,137,138,141,142,143,144,157,158,159,203,204,207,208,209,210,223,224,225]
# 2クラス合同 小石川1,2年 2CD合同あり用
#j = [4,5,6,9,10,11,12,22,23,24,25,26,27,70,71,72,75,76,77,78,88,89,90,91,92,93,137,138,141,142,143,144,157,158,159]
# 2クラス合同 小石川1,2年 2ABCD合同あり用
#j = [4,5,6,9,10,11,12,22,23,24,25,26,27,70,71,72,75,76,77,78,88,89,90,91,92,93]
# 2クラス合同 小石川1,2年 道徳以外合同あり用
#j = []

#joint = [[i, i+33] for i in j]
# 2クラス合同 小石川1,2年 道徳1,2年ごと合同あり用
#joint = [[32,213]]
#joint = [[32,213], [245,335],[334,336]]

#j = [0, 33, 52, 84]
#joint = list(itertools.combinations(j, 2))


#joint = [[4,37],[5,38],[6,39],[9,42],[10,43],[11,44],[12,45],[22,55],[23,]]
joint = [[],[]]
#総合
#joint = [[[22,45],[45,68],[68,89],[89,110],[110,131],[131,152],[152,175],[175,198],],[]]

In [13]:
# 第7項
# 同じ時間帯のコマ
#gen_list = [[0,3,6], # 1限
#            [1,4,7], # 2限
#            [2,5,8], # 3限
#]
gen_list = [
   [0,5,10,15,20], # 1限
   [3,8,13,18,23], # 4限
]
# # 1,5限
# gen_list = [[0,7,14,19,26],[4,11,18,23,30]]
# 1日1コマのみ入ってほしい科目群
# ver2:[Ja1(ID:0), Ja2(ID:1), Ja3(ID:2)], [Ea1(ID:6), Ea2(ID:7)], ...
#one_per_gen = [[0,1,2], [3,4,5], [6,7],
#               [9,10,11], [12,13,14], [15,16],
#               [18,19,20], [21,22,23], [24,25],
#              ]
# class4:◯◯は1,4限に固まって欲しくない
#one_per_gen = [[
#     [10,11,12,13,14],[35,36,37,38,39],[59,60,61,62,63],[84,85,86,87,88], # 英語*4クラス
#     [0,1,2,3,4],[25,26,27,28,29],[49,50,51,52,53],[74,75,76,77,78], # 国語*4クラス
#     [5,6,7,8,9],[30,31,32,33,34],[54,55,56,57,58],[79,80,81,82,83], # 数学*4クラス
#    [15,16,17,18],[40,41,42,43],[64,65,66,67],#[89,90,91,92], # 理科*4クラス
#     [19,20,21,22],[44,45,46,47],[68,69,70,71],[93,94,95,96], # 社会*4クラス
    #[23],[72], # 体育*合同2クラス
#]]
# one_per_gen = [
#     [0,1,2,3],[33,34,35,36],[52,53,54,55],[84,85,86,87], # 1年国語
#     [103,104,105,106,107],[135,136,137,138,139],[158,159,160,161,162],[190,191,192,193,194], # 2年国語
#     [4,5,6,7,8],[4,5,6,37,38],[56,57,58,59,60],[56,57,58,88,89], # 1年数学
#     [108,109,110,111],[108,109,140,141],[163,164,165,166],[163,164,195,196], # 2年数学
#     [9,10,11,12,13],[9,10,11,12,39],[61,62,63,64,65],[61,62,63,64,90], # 1年英語
#     [112,113,114,115,116],[112,113,114,115,142],[167,168,169,170,171],[167,168,169,170,197], # 2年英語
# ]
#5教科 国語,数学,生命科学,コミュ英,英語表現
one_per_gen = [[
    [0,1,2],[4,5,6,7],[8,9,10],[14,15,16],[17,18],
    [23,24,25],[27,28,29,30],[31,32,33],[37,38,39],[40,41],
    [46,47,48],[50,51,52,53],[54,55,56],[60,61,62],[63,64],
    [69,70,71],[73,74,75,76],[77,78,79],[81,82,83],[84,85],
    [90,91,92],[94,95,96,97],[98,99,100],[102,103,104],[105,106],
    [111,112,113],[115,116,117,118],[119,120,121],[123,124,125],[126,127],
    [132,133,134],[136,137,138,139],[140,141,142],[144,145,146],[147,148],
    [153,154,155],[157,158,159,160],[161,162,163],[167,168,169],[170,171],
    [176,177,178],[180,181,182,183],[184,185,186],[190,191,192],[193,194],]
    ,[],
]
#one_per_gen = [[],[]]

In [11]:
# 第8項：1教員が3コマ連続にならないようにする 
"""
7/13 追加 renzoku_2koma
例えば、国語1→国語2は許容しないが、国語2→国語1は許容するという風にすれば、
2時間連続NGが適度に守られて、連続3時間は滅多になくなるのでは？という発想
"""
renzoku_3koma = [ # 3*3
    [0,1,2],[3,4,5],[6,7,8]
]
renzoku_2koma = [[0,1],[1,2],[3,4],[4,5],[6,7],[7,8]]
renzoku_2koma = [[0,1],[1,2],[3,4],[5,6],[6,7],[8,9],[10,11],[11,12],[13,14],[15,16],[16,17],[18,19],[20,21],[21,22],[23,24]]

"""renzoku_2koma = [ # class4
    [0,1],[1,2],[3,4],
    [5,6],[6,7],[8,9],
    [10,11],[11,12],[13,14],
    [15,16],[16,17],[18,19],
    [20,21],[21,22],[23,24],
]"""
# 連続3コマにしたくない先生
# 時間割データnot3renzokuのa先生が3連続で授業に入らない
not_renzoku_ID = [[0,1,2,3,4,5],  # a先生の授業ID
                  [9,10,11,12,13,14],  # 先生の授業ID
                  [6,7,8,15,16,17],  # 先生の授業ID
                  [18,19,20,21,22,23],  # 先生の授業ID
                 ]
# class4:
"""not_renzoku_ID = [
    [0,1,2,25,26,27,49,50,51,74,75,76], # 佐藤
    [5,6,7,30,31,32,54,55,56,79,80,81], # 狩野
    [10,11,12,35,36,37,59,60,61,84,85,86], # アリス
    [15,16,17,18, 40,41,42,43,64,65,66,67,89,90,91,92], # 小野
    [19,20,21,22,44,45,46,47,68,69,70,71,93,94,95,96], # 佐田
]"""

#class3
not_renzoku_ID = [[
    [0,1,2,25,26,27,49,50,51], # 佐藤
],
[],
]
#基本同じ先生は３連続にしないようにする
not_renzoku_ID = [[],[]]

In [15]:
import time
# --- 問題設定 ---
# コマ
#"""
# 3*3のとき
table = [[0, 1, 2],[3, 4, 5],[6, 7, 8]]

# 5*5のとき
table = [[0, 1, 2, 3, 4],[5, 6, 7, 8, 9],[10, 11, 12, 13, 14],[15, 16, 17, 18, 19],[20, 21, 22, 23, 24]]
# 小石川
#table = [[0,1,2,3,4,5,6],[7,8,9,10,11,12,13],[14,15,16,17,18],[19,20,21,22,23,24,25],[26,27,28,29,30,31,32]]

# 1週間のコマ数
weekly = 0
for day in table:
    for koma in day:
        if koma != None:
            weekly += 1

"""
注意：以上の変数は、１日あたりのコマ数が全日同じことを想定
その条件を変えると以下は書き換えの必要あり
"""
#学年の数
gradenum = 2

koma_datalist=[]

for gnum in range(gradenum):
    # 授業の情報を表すExcelファイルからadjacentを作る
    input_file = '0{}kawa.xlsx'.format(gnum+1)#読み込みファイルの名前をinput0.xlsxなどにする
    cell_list = CellList(input_file, sheet_num=0) 
    class_dict = ClassDict(cell_list)
    total = len(class_dict)
    adjacent = Adjacent(class_dict)

    preprocessing=[]
    
    #前の学年のデータ  
    if gnum!=0:
        for num in range(gnum-1):
            input_file = 'input{}.xlsx'.format(num)
            cell_list2 = CellList(input_file, sheet_num=0) 
            class_dict2 = ClassDict(cell_list2)
            preprocessing=Preprocessing(preprocessing,koma_datalist[num],class_dict,class_dict2)


    # 制約項の重み
    w0=1.0
    w1=3.0
    w2=6.0
    w3=1.0
    w4=1.0
    w5=0.5
    w6=3.0
    w7=0.5
    w8=2.0
    # ハミルトニアンのQUBO形式
    Q, constant = Hamiltonian(weekly, total, gnum,
                    preprocessing,
                    adjacent, 
                    convenience, 
                    renzoku_koma, renzoku_ID,
                    table, one_per_day,
                    joint,
                    gen_list, one_per_gen,
                    renzoku_2koma, not_renzoku_ID,
                    w0=w0,w1=w1, w2=w2, w3=w3, w4=w4, w5=w5, w6=w6, w7=w7, w8=w8)
    # ソルバーの設定：シミュレーテッドアニーリング
    solver = neal.SimulatedAnnealingSampler()
    # ソルバーによって解を求める
    start = time.time()
    response = solver.sample_qubo(Q, num_sweeps=100000, num_reads=10)
    end = time.time()
    print('SAにかかった時間 = ' + str((end-start)/60) + "\n")
    # 解とエネルギーを取り出す
    sheet_num = 0
    sample0, energy0 = next(response.data(fields=['sample', 'energy']))
    #for sample0, energy0 in response.data(fields=['sample', 'energy']):
    # 授業ID(i)とコマ(a)を解の情報から取り出す
    # koma_data = {ID : コマ}
    koma_data = {}
    for key, val in sample0.items():
        if val == 1:
            i = key // weekly
            a = key % weekly
            koma_data[i] = a
    # TODO: koma_dataもKomaData()みたいな関数にする
    print(str(len(koma_data)) + 'bit')
    koma_datalist.append(koma_data)

    # 計算結果の判定

    constraint(adjacent, total, convenience[gnum], renzoku_ID[gnum], renzoku_koma, one_per_day[gnum],one_per_gen[gnum], joint[gnum], table, not_renzoku_ID[gnum], renzoku_3koma, koma_data)
    output_file = 'out{0}_{1}.xlsx'.format(gnum,sheet_num)
    MakeExcelFile(output_file, class_dict, koma_data, table, weekly)
    sheet_num += 1
    

SAにかかった時間 = 2.913791294892629

209bit
成功
SAにかかった時間 = 3.4681958675384523

211bit
成功
